In [4]:
# 加入lumapi路径
import imp
#import os
lumapi = imp.load_source('lumapi', "C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py")
#os.add_dll_directory("C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py")
import lumapi

import matplotlib.pyplot as plt
import numpy as np
import time

from rich.progress import Progress


In [5]:
R=[]
r_crosses=np.array([])
for i in range(20):
    r_crosses=np.append(r_crosses,100e-6+i*2e-6)

In [6]:
# 打开fdtd仿真文件
fdtd = lumapi.FDTD(filename="fiber_bragg_grating.fsp")


In [7]:
with Progress() as progress:
    task_fdtd=progress.add_task("[cyan]Running FDTD...",total=len(r_crosses))
    for r_cross in r_crosses:
        # 需要切换到layout模式才可以更改参数
        fdtd.switchtolayout()
        # # 选中平面波源
        # fdtd.select("PlaneSource")
        # # 更改中心波长
        # fdtd.set("center wavelength",1.5e-6)
        fdtd.select("fiber_bragg_grating")
        fdtd.set("r_cross",r_cross)
        # 运行仿真
        fdtd.run()
        # 读取结果 第一个参数为监视器名称，第二个为结果名称
        R.append(fdtd.getresult("R","T"))
        progress.update(task_fdtd,advance=1)


Output()

In [9]:
Return_T=np.array(R[0]['T'])
for i in range(1,len(r_crosses)):
    Return_T=np.vstack((Return_T,R[i]['T']))
    # #Return_T.append(R[i]['T'])
    # wavelength=R[i]['lambda'][:,0]

    # fig=plt.figure(figsize=(10,10))
    # plt.subplot(1,1,1)
    # plt.plot(wavelength,Return_T[:,i])

In [10]:
np.savetxt('T.txt',Return_T)
np.savetxt('r_cross.txt',r_crosses)